In [ ]:
import torch
import json

from libraries.model   import nGCNN, eGCNN, diffuse, denoise, interpolate_graphs
from libraries.dataset import revert_standardize_dataset
from libraries.graph   import POSCAR_graph_encoding

# Checking if pytorch can run in GPU, else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# Instead of generating new graphs from random noise, we interpolate
# already-existing graphs in noisy space and denoise them
# A target property can be seeked with this approach as well

In [ ]:
# Define folder in which all data will be stored
target_folder    = 'models/GM_BiSI'
edge_model_name = f'{target_folder}/edge_model.pt'
node_model_name = f'{target_folder}/node_model.pt'

# Number of graphs to predict
N_predictions = 10

# Amount of noise for the generative process
sigma = 0.1

# Define target to be generated
target_tensor = None

# Load model data

In [ ]:
# Read the file in JSON format to a dictionary
with open(f'{target_folder}/model_parameters.json', 'r') as json_file:
    model_parameters = json.load(json_file)

# Number of diffusing and denoising steps
n_t_steps = model_parameters['n_t_steps']

# Decay of parameter alpha
noise_contribution = model_parameters['noise_contribution']
alpha_decay = 0.5 * (1 - noise_contribution**2)

# Dropouts for node and edge models (independent of each other)
dropout_node = model_parameters['dropout_node']
dropout_edge = model_parameters['dropout_edge']

# Generation of graph database for training

Load the datasets, already standarized if possible.

In [ ]:
labels_name         = f'{target_folder}/labels.pt'
dataset_name        = f'{target_folder}/dataset.pt'
dataset_name_std    = f'{target_folder}/standardized_dataset.pt'
parameters_name_std = f'{target_folder}/standardized_parameters.json'  # Parameters for rescaling the predictions

# Load the standardized dataset, with corresponding labels and parameters
dataset = torch.load(dataset_name_std)
labels  = torch.load(labels_name)

# Read the file in JSON format to a dictionary
with open(parameters_name_std, 'r') as json_file:
    dataset_parameters = json.load(json_file)

In [ ]:
# Calculate the mean and standard deviation of the number of nodes
total_nodes = torch.tensor([data.num_nodes for data in dataset])
mean_nodes  = torch.mean(total_nodes.float()).item()
std_nodes   = torch.std(total_nodes.float()).item()

mean_nodes, std_nodes

# Loading the model

In [ ]:
# Determine number of features in dataset, considering the t_step information
n_features = dataset[0].num_node_features + 1

# Instantiate the models for nodes and edges
node_model = nGCNN(n_features, dropout_node).to(device)
node_model.load_state_dict(torch.load(node_model_name))
node_model.eval()

edge_model = eGCNN(n_features, dropout_edge).to(device)
edge_model.load_state_dict(torch.load(edge_model_name))
edge_model.eval()

print('\nNode GCNN:')
print(node_model)
print('\nEdge GCNN:')
print(edge_model)

# Interpolating known cystals

In [ ]:
## Diffuse the initial dataset
dataset_t = []
for graph in dataset:
    # Diffuse the graph with some noise
    #print()
    #print('Diffusing...')
    graph_t, _ = diffuse(graph, n_t_steps,
                         s=alpha_decay)
    dataset_t.append(graph_t)

# Get random number of nodes
n_nodes = int(np.random.normal(mean_nodes, std_nodes))  ???

In [ ]:
# Interpolate the noisy version of the dataset into a new, diffused graph
diffused_graph = interpolate_graphs(dataset_t)  # Does not work so far

# Interpolation loop
predicted_dataset = []
with torch.no_grad():
    for i in range(N_predictions):
        # Denoise the diffused graph
        #print(f'Denoising...')
        denoised_graph, _ = denoise(diffused_graph, n_t_steps, node_model, edge_model,
                                    s=alpha_decay, sigma=sigma, target=target_tensor)
        
        # Append generated graph
        predicted_dataset.append(denoised_graph)

# Revert standardization
denoised_graphs = revert_standardize_dataset(predicted_dataset, dataset_parameters)
denoised_graphs

In [ ]:
for i in range(N_predictions):
    print()
    print(i+1)
    graph = denoised_graphs[i].clone()
    try:
        POSCAR_graph_encoding(graph, file_name=f'POSCAR-{i}', POSCAR_directory='./')
    except SystemExit:
        continue